In [94]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('VI_train.csv', index_col='id').iloc[:,1:]
df_test = pd.read_csv('VI_test.csv', index_col='id')

In [157]:
X = df.iloc[:,:-1]
y = df.iloc[:, -1]
X_test = df_test

GENDER_MAPPING = {'Male': 0, 'Female': 1}
VEHICLE_AGE_MAPPING = {'< 1 Year': 0, '1-2 Year': 1, '> 2 Years': 2}
VEHICLE_DAMAGE = {'Yes': 0, 'No': 1}
X = X.replace({'Gender': GENDER_MAPPING, 'Vehicle_Age': VEHICLE_AGE_MAPPING, 'Vehicle_Damage':VEHICLE_DAMAGE})
X_test = X_test.replace({'Gender': GENDER_MAPPING, 'Vehicle_Age': VEHICLE_AGE_MAPPING, 'Vehicle_Damage':VEHICLE_DAMAGE})

X_train, X_valid, y_train, y_valid = train_test_split(X,y,
                                                      test_size=0.25,
                                                      random_state=33)

In [103]:
def evaluate(clf, X_valid, y_valid):
    y_pred = clf.predict(X_valid)
    y_true = y_valid
    precision = sum([i and j for i, j in zip(y_true, y_pred)]) / sum(y_pred)
    recall = sum([i and j for i, j in zip(y_true, y_pred)]) / sum(y_true)
    f1_score = 2 * (recall * precision) / (recall + precision)
    print('f1_score =', f1_score)
    return f1_score

In [172]:
import json
def submit(cls, X_test):
    y_pred = clf.predict(X_test)
    with open('submission.json', 'w') as f:
        json.dump({str(k):int(v) for k,v in zip(X_test.index, y_pred)}, f, indent=2)

In [167]:
import numpy as np
X_train_balance = pd.concat([X_train[y_train == 0][:27727], X_train[y_train == 1]])
y_train_balance = pd.Series(np.concatenate([np.zeros(X_train[y_train == 0][:27727].shape[0]), 
                                            np.ones(X_train[y_train == 1].shape[0])]))

In [168]:
# 决策树： F1 score = 0.2937212536100117
# X_train 欠采样：F1 score = 0.38390311296628354
if True:
    from sklearn import tree
    clf = tree.DecisionTreeClassifier(criterion='entropy')
    clf = clf.fit(X_train_balance, y_train_balance)
    # clf.score(X_valid, y_valid)
    evaluate(clf, X_valid, y_valid)
    submit(clf, X_test)

f1_score = 0.38390311296628354


In [176]:
# AdaBoost 欠采样 0.40685683674835943
if True:
    from sklearn.ensemble import AdaBoostClassifier
    from sklearn import tree
    clf = AdaBoostClassifier(tree.DecisionTreeClassifier(criterion='entropy'))
    clf = clf.fit(X_train_balance, y_train_balance)
    evaluate(clf, X_valid, y_valid)

f1_score = 0.40821073128515795


In [188]:
# EasyEnsembleClassifier + DecisionTreeClassifier: 0.4287806664225558
if True:
    from imblearn.ensemble import EasyEnsembleClassifier     
    from sklearn import tree
    clf = EasyEnsembleClassifier(n_estimators=20,
                                 base_estimator=tree.DecisionTreeClassifier(criterion='entropy'),
                                 random_state=42)
    clf = clf.fit(X_train, y_train)
    evaluate(clf, X_valid, y_valid)
    submit(clf, X_test)

f1_score = 0.42760961191088737


In [191]:
# EasyEnsembleClassifier + DecisionTreeClassifier: 0.4287806664225558
if True:
    from imblearn.ensemble import EasyEnsembleClassifier     
    from sklearn import tree
    clf = EasyEnsembleClassifier(n_estimators=30,
                                 base_estimator=AdaBoostClassifier(tree.DecisionTreeClassifier(criterion='entropy')),
                                 random_state=42)
    clf = clf.fit(X_train, y_train)
    evaluate(clf, X_valid, y_valid)
    submit(clf, X_test)

f1_score = 0.42749720292457005
